# TØ 2

# Opgave 1

a) Løs numerisk fra $t=0$ til $t=4$ differentialligningen
\begin{equation*}
\frac{d^2y}{dt^2}-3\frac{dy}{dt}+12y=0
\end{equation*}
under betingelserne $y(0)=0.0298$ og $\displaystyle\left.\frac{dy}{dt}\right|_{t=0}=-3$.

b) Beregn værdien af $y$ til $t=4$. Du skal nok sætte
    `max_step` til $10^{-3}$ for
at få et nøjagtigt resultat.


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

def dpdt(t,p):
    y = p[0]
    dydt = p[1]
    d2ydt2 = 3 * dydt - 12 * y
    return [dydt, d2ydt2]

mysol = solve_ivp(dpdt, [0, 4], [0.0298, -3], max_step=1e-3)
mysol.y[0][-1]

# Opgave 2

To forskellige ramper starter begge i $(x,y)=(0,h)$, hvor $h=\displaystyle\frac{45}{2}$ og
går gennem punktet $(x,y)=(x_1,0)$, hvor $x_1=30$. Den ene rampe er
lineært faldende, mens den anden rampe er beskrevet ved en parabel,
der endvidere går gennem punktet $(x_0,0)$, $x_0=10$.

a) Opskriv funktionsudtryk for højdeprofilen af de to ramper, hhv $y = f_\mathrm{lin}(x)$
for den lineære rampe og $y = f_\mathrm{par}(x)$ for den parabel-formede
rampe. Bestem parametre i udtrykkene med SymPy (med mindre du
umiddelbart kan gennemskue hvad parametrene skal være).
Plot de to funktioner med python-kommandoer.

![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_01.png)

In [ ]:
import numpy as np
import sympy as sp
xsym = sp.symbols('x')
asym = sp.symbols('a')

x0 = 10
x1 = 30

hsym = 45/sp.Integer(2)
h = hsym.evalf()

fparsym = asym * (xsym - x0) * (xsym - x1)
fparsym.expand(), h

In [ ]:
fparsym.subs(xsym,0)

In [ ]:
a_sol = sp.solve(fparsym.subs(xsym,0) - hsym)[0]
fparsym_fixed = fparsym.subs(asym,a_sol)
fparsym_fixed

In [ ]:
fpar = sp.lambdify(xsym, fparsym_fixed, 'numpy')
flin = lambda x: h - h/x1*x

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font',size=16)

fig, ax = plt.subplots()
xs1 = np.linspace(0, 30, 100)
xs2 = np.linspace(30, 40, 30)
ax.plot(xs1, flin(xs1), label='$f_{lin}(x)$')
ax.plot(xs1, fpar(xs1), label='$f_{par}(x)$')
ax.plot(xs2, flin(xs2), linestyle='--', color='C0')
ax.plot(xs2, fpar(xs2), linestyle='--', color='C1')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.grid()
ax.legend()
ax.set_aspect('equal')

fig.tight_layout()
fig.savefig('lin_par_rampe_01.png')

Bevægelsen for et legeme på ramperne under indvirkning af
tyngdekraften er beskrevet ved følgende differentialligning:
\begin{equation*}
\frac{d^2x}{dt^2}=\frac{\displaystyle -\frac{dy}{dx}}{1+\left(\displaystyle\frac{dy}{dx}\right)^2}\Bigl(g+\frac{d^2y}{dx^2}\left(\frac{dx}{dt}\right)^2\Bigr),
\end{equation*}
hvor $g=9.8$ er tyngdeaccelerationen og hvor $y(x)$ er rampens højdeprofil som funktion af $x$. For den lineære
rampe er hældningen, $\displaystyle\frac{dy}{dx}$, en konstant for alle
værdier af $x$ mens den anden afledede, $\displaystyle\frac{d^2y}{dx^2}$, er
nul. For den parabel-formede rampe er hældningen en funktion af $x$ og
den anden afledede en
konstant. 
Differentialligningen
kan synes stor og kompliceret, men læg mærke til at når $y$ er en
kendt funktion af $x$, så er det eneste
ubekendte, der er tilbage i ligningen, de to hhv 1.- og
2.-ordens tidsafledede af $x$-positionen, her markeret med rødt:
\begin{equation*}
\color{red}{\frac{d^2x}{dt^2}}=\frac{\displaystyle -\frac{dy}{dx}}{1+\left(\displaystyle\frac{dy}{dx}\right)^2}\Bigl(g+\frac{d^2y}{dx^2}\left(
\color{red}{\frac{dx}{dt}}\right)^2\Bigr).
\end{equation*}
Differentialligningen har altså samme form som dem, der blev løst
til forelæsningen
og kan dermed løses på
samme måde.



b) Udled med SymPy (eller pen og papir, hvis det er meget nemt)
de omtalte udtryk for hældning og den anden afledede. Løs nu
differentialligningen med
`solve_ivp` for tider i intervallet $0\le t\le
4$ og plot $x(t)$ for hver af de to ramper så du kan afgøre for hvilken
rampe, et legeme startet stilleliggende ud i $\bigl(0,f(0)\bigr)$
hurtigst muligt vil passere $x=x_1$.
![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_02.png)
Når $x$ først er fundet som funktion af $t$ kan man
efterfølgende bestemme $y(t)$ ud fra den geometriske betingelse at
legemet befinder sig på rampen, dvs.
\begin{equation*}
y(t) = f\big(x(t)\big),
\end{equation*}
hvor $f(x)$ er $f_\mathrm{lin}(x)$ eller $f_\mathrm{par}(x)$ alt efter hvilken rampe, der er tale om.


In [ ]:
flin_prime = - h/x1

fpar_prime_sym = sp.diff(fparsym_fixed,xsym)
fpar_prime = sp.lambdify(xsym, fpar_prime_sym, 'numpy')
fpar_pp_sym = sp.diff(fpar_prime_sym,xsym)
fpar_pp = sp.lambdify(xsym, fpar_pp_sym, 'numpy')

In [ ]:
from scipy.integrate import solve_ivp
g = 9.8
trange = [0, 4]
x0 = 0
v0 = 0
q0 = [x0, v0]
def dqlindt(t, q):
    x = q[0]
    v = q[1]
    dxdt = v
    dvdt = -flin_prime/(1+flin_prime**2)*g
    return [dxdt, dvdt]

def dqpardt(t, q):
    x = q[0]
    v = q[1]
    dxdt = v
    dvdt = -fpar_prime(x)/(1+fpar_prime(x)**2)* \
                        (g + fpar_pp(x)*v**2)
    return [dxdt, dvdt]

solve_lin = solve_ivp(dqlindt, trange, q0, max_step=1e-3)
ts_lin = solve_lin.t
xs_lin = solve_lin.y[0]

solve_par = solve_ivp(dqpardt, trange, q0, max_step=1e-3)
ts_par = solve_par.t
xs_par = solve_par.y[0]

fig, ax = plt.subplots()
ax.plot(ts_lin, xs_lin, label='via $y=f_{lin}(x)$')
ax.plot(ts_par, xs_par, label='via $y=f_{par}(x)$')
ax.plot([0, 4], [x1, x1], linestyle='dashed')
ax.legend()
ax.grid()
ax.set_xlabel('$t$')
ax.set_ylabel('$x$')

fig.tight_layout()
fig.savefig('lin_par_rampe_02.png')

c) Løs nu differentialligningen igen for hver af de to ramper. Sørg
for at få de to løsninger til de samme ækvidistante tider.
Udarbejd derefter en update-funktion og test den:


![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_03.png)
![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_04.png)
![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_05.png)


Foretag en
animation af den samtidige bevægelse af to legemer,
der startes på hver deres rampe.

![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_06.gif)



In [ ]:
fig, ax = plt.subplots()
xs1 = np.linspace(0, x1, 100)
xs2 = np.linspace(x1, 40, 30)
ax.plot(xs1, flin(xs1), label='$f_{lin}(x)$')
ax.plot(xs1, fpar(xs1), label='$f_{par}(x)$')
ax.plot(xs2, flin(xs2), linestyle='--', color='C0')
ax.plot(xs2, fpar(xs2), linestyle='--', color='C1')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.grid()
ax.legend()
ax.set_aspect('equal')

In [ ]:
t_eval = np.linspace(0, trange[1], 100)

solve_lin = solve_ivp(dqlindt, trange, q0, max_step=1e-3, t_eval=t_eval)
xs_lin = solve_lin.y[0]

solve_par = solve_ivp(dqpardt, trange, q0, max_step=1e-3, t_eval=t_eval)
xs_par = solve_par.y[0]

ball_lin = ax.plot([],[],'o', color='k')[0]
ball_par = ax.plot([],[],'o', color='C3')[0]

fig

In [ ]:
def update(i):
    ball_lin.set_data(xs_lin[i],flin(xs_lin[i]))
    ball_par.set_data(xs_par[i],fpar(xs_par[i]))
    return [ball_lin, ball_par]

In [ ]:
update(30)
fig.tight_layout()
fig.savefig('lin_par_rampe_03.png')
fig

In [ ]:
update(60)
fig.tight_layout()
fig.savefig('lin_par_rampe_04.png')
fig

In [ ]:
update(83)
fig.tight_layout()
fig.savefig('lin_par_rampe_05.png')
fig

In [ ]:
from matplotlib import animation
plt.rc('animation', html='jshtml')


Nframes = len(t_eval)
anim = animation.FuncAnimation(fig,
                               update,
                               frames=Nframes,
                               interval=10,
                               blit=True)
anim

In [ ]:
writer = animation.writers['pillow'](fps=25) # frames per second
anim.save('lin_par_rampe_06.gif', writer=writer)

d) Fokusér nu på den parabel-formede rampe og lad den være defineret
for $0\le x\le b$, hvor $b=40$. Bestem et legemes bevægelse på rampen
i tiden $0\le t\le 23$ og notér dig at den bliver oscillatorisk. Idet
legemes masse sættes til $m=1/7$ skal du ud fra outputtet fra
`solve_ivp` beregne og plotte den kinetiske og den
potentielle energi af legemet som funktion af tiden.
Læg mærke til at $\displaystyle\frac{dx}{dt}$ *ikke* er hastigheden $v$ af
legemet. Det er kun $x$-komposanten af legemets hastighed. Da legemet er
tvunget til at bevæge sig ad rampen vil dets hastighedsvektor
imidlertid altid pege langs med rampen, der har retningsvektoren:
\begin{equation*}
\hat{\mathbf{s}} = \frac{1}{\sqrt{1+\left(\displaystyle\frac{dy}{dx}\right)^2}}\left[\begin{array}{c}1\\ \displaystyle\frac{dy}{dx}\end{array}\right],
\end{equation*}
hvorfor legemets hastighedsvektor kan skrives:
\begin{equation*}
\mathbf{v} = v\hat{\mathbf{s}}.
\end{equation*}
Med den sædvanlige måde at skrive hastighedsvektoren på:
\begin{equation*}
\mathbf{v} = 
  \left[\begin{array}{c}\displaystyle\frac{dx}{dt}\\ \\
        \displaystyle\frac{dy}{dt}\end{array}\right]
\end{equation*}
kan man ud fra $x$-komposanterne udlede dette udtryk:
\begin{equation*}
v = \displaystyle\frac{dx}{dt}\sqrt{1+\left(\displaystyle\frac{dy}{dx}\right)^2}.
\end{equation*}
Verificér at den mekaniske energi er bevaret ved at lave dette plot af
potentiel, kinetisk og mekanisk energi:

![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_07.png)

Bestem endvidere den maksimale kinetiske energi
med `np.max`-kommandoen på listen af beregnede
kinetiske energier. Udskriv værdien.


In [ ]:
b = 40
tmax = 23
trange = [0, tmax]
t_eval = np.linspace(0,tmax,1000)
m = 1/7

solve_par = solve_ivp(dqpardt, trange, q0, max_step=1e-3, t_eval=t_eval)
ts = solve_par.t
xs = solve_par.y[0]
vxs = solve_par.y[1]

vs = vxs * np.sqrt(1 + fpar_prime(xs)**2)
pots = m * g * fpar(xs)
kins = 1/2 * m * vs**2
fig, ax = plt.subplots()
ax.plot(ts, pots, label='P')
ax.plot(ts, kins, label='K')
ax.plot(ts, pots + kins, label='P+K')
ax.set_xlabel('$t$')
ax.set_ylabel('Energy')
ax.legend()
fig.tight_layout()
fig.savefig('lin_par_rampe_07.png')
max(kins)

e) Lav evt en animation af den oscillatoriske bevægelse. 

![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_08.gif)

In [ ]:
fig, ax = plt.subplots()
xs = np.linspace(0, b, 100)
ax.plot(xs, fpar(xs), color='C1')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.grid()
ax.set_aspect('equal')
ball_par = ax.plot([],[],'o', color='C3')[0]

t_eval = np.linspace(0,tmax,200)
solve_par = solve_ivp(dqpardt, trange, q0, max_step=1e-3, t_eval=t_eval)
xs = solve_par.y[0]

def update(i):
    ball_par.set_data(xs[i],fpar(xs[i]))
    return [ball_par]

In [ ]:
update(0)
fig

In [ ]:
from matplotlib import animation
plt.rc('animation', html='jshtml')


Nframes = len(xs)
anim = animation.FuncAnimation(fig,
                               update,
                               frames=Nframes,
                               interval=20,
                               blit=True)
anim

In [ ]:
writer = animation.writers['pillow'](fps=25) # frames per second
anim.save('lin_par_rampe_08.gif', writer=writer)

f) Vi vender nu tilbage til spørgsmålet om hvornår legemerne passerer
$x_1$ når de startes ud i $(0,h)$. Man kan lave en hel serie af
forskellige parabel-formede ramper, hvis $x_0$ betragtes som en parameter.
Her er fire ramper, der alle starter i $(0,h)$ og går gennem $(x_0,
0)$ og $(x_1,0)$:

![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_09.png)

Bemærk hvordan $x_0=1000$ næsten bliver til en ret linie. Lav et
tilsvarende plot af nogle parabler for forskellige $x_0$-værdier.



In [ ]:
fig, ax = plt.subplots()
xs = np.linspace(0, b, 100)
for x0 in [10, 20, 30, 1000]:
    foptsym = hsym/x0/x1 * (xsym - x0) * (xsym - x1)
    fopt = sp.lambdify(xsym,foptsym,'numpy')
    ax.plot(xs, fopt(xs), label='$x_0=' + str(x0) + '$')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_aspect('equal')
ax.legend()
ax.grid()
fig.tight_layout()
fig.savefig('lin_par_rampe_09.png')

g) Skriv nu en funktion, der givet $x_0$:
- med SymPy bestemmer funktionsforskriften for parablen, samt dennes
  1. og 2. afledede.
- konverterer SymPy-funktionerne til NumPy-funktioner.
- løser differentialligningen
  frem til legemet passerer $x_1$. Dette håndteres som en hændelse,
  der stopper
  løsningen af differentialligningen.
- returnerer tidspunktet for ankomsten til $x_1$.


Testkør funktionen med $x_0=10$ og $x_0=1000$, der svarer til den oprindelige
parabelformede rampe samt approximerer den lineære rampe. Fra figuren i
spørgsmål b) kan du aflæse at du skal få ca. $3.45$ og $3.55$. Hvad får du?


In [ ]:
N = 12
x0vals = np.logspace(1,3,N)

at_x1 = lambda t, q: q[0] - x1
at_x1.terminal = True

def time_to_x1(x0):
    foptsym = hsym/x0/x1 * (xsym - x0) * (xsym - x1)

    fopt_prime_sym = sp.diff(foptsym,xsym)
    fopt_prime = sp.lambdify(xsym, fopt_prime_sym, 'numpy')
    fopt_pp_sym = sp.diff(fopt_prime_sym,xsym)
    fopt_pp = sp.lambdify(xsym, fopt_pp_sym, 'numpy')
    
    def dqoptdt(t, q):
        x = q[0]
        v = q[1]
        dxdt = v
        dvdt = -fopt_prime(x)/(1+fopt_prime(x)**2)* \
                            (g + fopt_pp(x)*v**2)
        return [dxdt, dvdt]

    solve_opt = solve_ivp(dqoptdt, trange, q0, max_step=1e-2, events=[at_x1])
    ts_opt = solve_opt.t
    return ts_opt[-1]

time_to_x1(10),time_to_x1(1000)

h) Scan nogle værdier af $x_0$ og plot en kurve over den tid det tager
at nå frem til $x_1$ som funktion af $x_0$.

![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_10.png)


In [ ]:


tvals = []
for x0 in x0vals:
    tvals.append(time_to_x1(x0))

fig, ax = plt.subplots()
ax.plot(np.abs(x0vals), tvals)
ax.set_xlabel('$x_0$')
ax.set_ylabel('$t(x=x_1)$')
ax.grid()
fig.tight_layout()
fig.savefig('lin_par_rampe_10.png')

i) Benyt `fmin` til at bestemme
den optimale værdi af $x_0$, der bringer legemet hurtigst muligt frem
til $x_1$. Afsæt det fundne punkt i plottet:

![Image Title](https://python23.sci2u.dk/media/4/img/lin_par_rampe_11.png)

Forhold dig til at det ikke er den lineære rampe, der den
optimale. Kan du forklare hvorfor?

Hint, hvis du får en fejl med `fmin`: Når `fmin` søger kalder den funktionen med et
1-dimensionelt NumPy array med ét tal. Alt efter hvordan du bruger
dette tal i funktionen kan det gå galt. SymPy bryder sig ikke om et
sådant NumPy-array. Det kan derfor være en god idé at gøre således:
```
x0opt = fmin(lambda x0: time_to_x1(float(x0)), 10)[0]
```
hvor `time_to_x1` er din funktion fra spørgsmål g
og hvor `lambda`-funktionen bruges til at wrappe
den så `fmin`'s NumPy-array bliver konverteret til
en float.



In [ ]:
from scipy.optimize import fmin
x0opt = fmin(lambda x0: time_to_x1(float(x0)), 10)[0]
ax.scatter(x0opt,time_to_x1(x0opt))
fig.tight_layout()
fig.savefig('lin_par_rampe_11.png')
fig

# Opgave 3 (hvis du synes den lyder sjov)

Denne opgave knytter sig til
  rampe-opgaven ovenfor og handler om at 
udlede udtrykket for
  differentialligningen.

a) Start med at argumentere for at forholdet mellem den differentielle
tilvækst i vejlængden langs rampen og den differentielle tilvækst i
$x$-koordinaten på rampen er:
\begin{equation*}
\displaystyle\frac{ds}{dx}=
\sqrt{1+\left(\frac{dy}{dx}\right)^2}
\end{equation*}
b) Opskriv dernæst Newton II for et legeme på rampen:
\begin{equation*}
\begin{array}{rcl}
  m\color{blue}{\displaystyle\frac{d^2s}{dt^2}}&=&(0,-mg)\cdot \hat{\mathbf{s}}\\ \\
  &=& -mg\frac{\displaystyle\frac{dy}{dx}}{\sqrt{\displaystyle 1+\left(\displaystyle\frac{dy}{dx}\right)^2}}.
\end{array}
  \end{equation*}
du kan få mening i det ud fra denne figur:


![Image Title](https://python23.sci2u.dk/media/4/img/rampe_parabel_II_zoom.png)


c) Benyt denne kombination af kædereglen og produktreglen:
\begin{equation*}
\begin{array}{rcl}
\color{blue}{\displaystyle\frac{d^2s}{dt^2}}&=&\displaystyle\frac{d}{dt}\Bigl\{\displaystyle\frac{ds}{dx}\displaystyle\frac{dx}{dt}\Bigr\}\\
  \\
  &=&
\color{green}{\displaystyle\frac{d}{dt}\Bigl\{\displaystyle\frac{ds}{dx}\Bigr\}}\displaystyle\frac{dx}{dt}
      +\displaystyle\frac{ds}{dx}\displaystyle\frac{d}{dt}\Bigl\{\displaystyle\frac{dx}{dt}\Bigr\}\\ \\
                               &=&
                                   \big(\displaystyle\frac{dx}{dt}\big)^2
                                   \frac{\displaystyle\frac{dy}{dx}}{\sqrt{\displaystyle
                                   1+\left(\displaystyle\frac{dy}{dx}\right)^2}}
                                   \displaystyle\frac{d^2y}{dx^2}
                                   + \sqrt{\displaystyle                                                         
                                   1+\left(\displaystyle\frac{dy}{dx}\right)^2}\displaystyle\frac{d^2x}{dt^2}
                                   \end{array}
\end{equation*}

hvor den grønne del er blevet omskrivet på følgende vis:
\begin{equation*}
\begin{array}{rcl}
\color{green}{\displaystyle\frac{d}{dt}\Bigl\{\displaystyle\frac{ds}{dx}\Bigr\}}
  &=&\displaystyle\frac{dx}{dt}\displaystyle\frac{d}{dx}\Bigl\{\displaystyle\frac{ds}{dx}\Bigr\}\\ \\
  &=&\displaystyle\frac{dx}{dt}\displaystyle\frac{d}{dx}\sqrt{1+\bigl(\displaystyle\frac{dy}{dx}\bigr)^2}\\ \\
  &=&\displaystyle\frac{dx}{dt}\frac{1}{2\sqrt{1+\bigl(\displaystyle\frac{dy}{dx}\bigr)^2}}\cdot
      2\displaystyle\frac{dy}{dx}\displaystyle\frac{d^2y}{dx^2}
\end{array}
\end{equation*}
d) De to udtryk skrevet med blå kan nu sættes lig hinanden og du kan
udlede 
  differentialligningen.

# Opgave 4

Det oplyses om en funktion $y(x)$ at den minimerer dette funktional:
$$
I(y)=\displaystyle\int_0^1\left(\frac{dy}{dx} - y(x)\right)^2 dx,\quad y(0)=0,\quad y(1)=2,
$$
når $y(x)=y_0(x)$, hvor:
$$
y_0(x)=2\frac{\sinh(x)}{\sinh(1)}
$$

a) Beregn værdien $I(y_0)$

In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
plt.rc('font',size=20)
symx = sp.symbols('x')

In [ ]:
symy0 = 2 * sp.sinh(symx) / sp.sinh(1)
symdy0dx = sp.diff(symy0,symx)
symdy0dx

In [ ]:
a = sp.integrate((symdy0dx - symy0)**2,(symx,0,1))
float(a)

For at beregne $I(y)$ for andre funktioner, der opfylder grænsebetingelserne indføres:

$$
\Delta (x) = \sin(\pi x)
$$

b) Beregn $I(y_a)$, hvor:

$$
y_a(x) = y_0(x) + a \Delta (x)
$$

for værdier af $a$ mellem $-1.2$ og $1.2$. Plot $y_a(x)$ for de forskellige værdier af $a$ og benyt en farve, der afspejler værdien af $I(y_a)$.

![Image Title](https://python23.sci2u.dk/media/4/img/functional_analysis_101.png)

In [ ]:
deltay = sp.sin(symx*sp.pi)
deltay.subs(symx,0),deltay.subs(symx,0.5),deltay.subs(symx,1)

In [ ]:
fig, ax = plt.subplots()

numx = np.linspace(0,1,30)

norm = plt.Normalize(vmin=1.25, vmax=9)
cmap = plt.cm.coolwarm

for a in np.linspace(-1.2,1.2,11):
    symy = symy0 + a * deltay
    symdydx = sp.diff(symy,symx)
    vali = float(sp.integrate((symdydx - symy)**2,(symx,0,1)))
    print(a,vali)
    numy = sp.lambdify(symx,symy,'numpy')
    ax.plot(numx,numy(numx),color=cmap(norm(vali))) #f'C{int(vali)}')
    
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.2)
cbar.set_label('$I(y_a)$', fontsize=20)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.grid('on')

In [ ]:
fig.tight_layout()
fig.savefig('functional_analysis_101.png')